In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob

# Camera Calibration and Undistorting Images
In this notebook we are going to calibrate a monocular camera (i.e. estimate the camera matrix and distortion parameters) and use the calibration to undistort an image. We will start by using the standard OpenCV functions first and later implement the calibration and undistortion form scratch using numpy and scipy.
## Camera Calibration using OpenCV
I have printed out a [chessboard pattern](https://github.com/opencv/opencv/blob/master/doc/pattern.png) and took pictures of it using my phone's camera. For efficiency reasons I scaled down the images by a factor of 2. We will follow the [official OpenCV tutorial on camera calibration](https://docs.opencv.org/master/dc/dbb/tutorial_py_calibration.html).
<img src="data/IMG_1239.jpg" alt="drawing" width="30%"/>

In [2]:
# Extract circle centers for all images. This might take a while...
image_files = glob.glob('data/*.jpg')

grid_size = 25 # mm
grid_shape = (9,6)

image_corners = np.zeros((0, grid_shape[0]*grid_shape[1] ,2))
chessboard_positions = np.zeros((grid_shape[0]*grid_shape[1],3))
chessboard_positions[:,:2] = np.mgrid[0:grid_shape[0],0:grid_shape[1]].T.reshape(-1,2)
chessboard_positions *= grid_size
image_shape = np.array([])

for file in image_files:
    img = cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2GRAY)
    image_shape = img.shape
    ret, corners = cv2.findChessboardCorners(img, grid_shape)
    if ret:
        image_corners = np.append(image_corners, np.array(corners).reshape(1, -1, 2), axis=0)
    else:
        print(f"No chessboard detected in image {file}")
print(f"Processed {len(image_files)} images and found {image_corners.shape[0]} chessboards.")

Processed 29 images and found 29 chessboards.


In [3]:
def calibrate_camera_openCV():
    object_points = np.tile(chessboard_positions, (image_corners.shape[0], 1, 1))
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points.astype(np.float32), image_corners.astype(np.float32), image_shape[::-1], None, None)
    return mtx, dist[0]

camera_mat_cv, dist_cv = calibrate_camera_openCV()
print("Camera matrix returned by OpenCV:")
with np.printoptions(suppress=True, precision=2):
    print(camera_mat_cv)
print("Distortion coefficients returned by OpenCV:")
print(f"k1: {dist_cv[0]:+.3e}; \tk2: {dist_cv[1]:+.3e}; \tp1: {dist_cv[2]:+.3e}; \tp2: {dist_cv[3]:+.3e}")

Camera matrix returned by OpenCV:
[[1534.97    0.   1010.01]
 [   0.   1558.1   747.42]
 [   0.      0.      1.  ]]
Distortion coefficients returned by OpenCV:
k1: +2.213e-01; 	k2: -9.994e-01; 	p1: -2.081e-05; 	p2: +4.839e-05


## Camera Calibration from Scratch
### Estimating the Camera Matrix
In this section we are going to implement the camera calibration procedure inspired by [Zhang '98](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr98-71.pdf). 
We write down the perspective projection equation for a planar grid. The origin of the world is therefore defined by the chessboardd pattern. Note that all z-coordinates are zero because all world points lie on the plane. 

$$ 
\lambda \left( \begin{array}{r} u \\ v \\ 1 \end{array}\right) = K 
\left[ \begin{array}{c c} R_{cw} &  T_{cw} \end{array}\right] 
\left( \begin{array}{c} X_w \\ Y_w \\ 0 \\ 1\end{array}\right) = 
\underbrace{K \left[ \begin{array}{c c } r_1 & r_2 & t\end{array}\right]}_{H} 
\left( \begin{array}{c} X_w \\ Y_w \\ 1\end{array}\right)
$$
The matrix $H$ is the same for all corners in one image and can be estimated using for example the **direct linear transform (DLT)**. This however assumes that all elements in $H$ are independent which in in general not the case. We will only use the DLT only as a first guess and refine the estimation later on using all constraints.

$$ 
\lambda \left( \begin{array}{r} u \\ v \\ 1 \end{array}\right) = 
\left( \begin{array}{ccc}
    h_{11} & h_{12} & h_{13}\\ h_{21} & h_{22} & h_{23} \\ h_{31} & h_{32} & h_{33}
\end{array}\right) 
\left( \begin{array}{c} X_w \\ Y_w \\ 1\end{array}\right) = 
\left( \begin{array}{c}h_1^T \\ h_2^T \\ h_3^T\end{array}\right) P 
\Rightarrow 
\left\{
\begin{array}{l l}
    u = \frac{\lambda u}{\lambda} = \frac{h_1^T \cdot P}{h_3^T \cdot P} \\ 
    v = \frac{\lambda v}{\lambda} = \frac{h_2^T \cdot P}{h_3^T \cdot P}
\end{array}\right.
\Rightarrow 
\left\{
\begin{array}{l l}
    (h_1^T - u h_3^T)\cdot P=0 \\ 
    (h_2^T - v h_3^T)\cdot P=0 
\end{array}\right.
\Rightarrow 
\left( \begin{array}{ccc}
P^T & 0^T & -uP^T \\0^T & P^T & -vP^T
\end{array}\right) 
\left( \begin{array}{c}h_1 \\ h_2 \\ h_3\end{array}\right) =0 $$

This reasoning can be applied to all n corners in one image and the resulting equations can be stacked to yield one big matrix. The matrix $Q$ can be constructed using the observed corners. The least square solution for $H$ can be found with the **Singular Value Decomposition (SVD)** ([Wikipedia](https://en.wikipedia.org/wiki/Singular_value_decomposition#Solving_homogeneous_linear_equations)).
$$ \underbrace{\left( \begin{array}{ccc}
P_1^T & 0^T & -u_1P_1^T \\
0^T & P_1^T & -v_1P_1^T \\ 
P_2^T & 0^T & -u_2P_2^T \\
0^T & P_2^T & -v_2P_2^T \\
\vdots & \vdots & \vdots \\
P_n^T & 0^T & -u_nP_n^T \\
0^T & P_n^T & -v_nP_n^T \end{array} \right)}_{Q\in\mathbb{R}^{2n \times 9}} \left( \begin{array}{c}h_1 \\ h_2 \\ h_3\end{array}\right) = Q \cdot H' = 0$$

In [4]:
def calculate_H_DLT(corners, chessboard_positions):
    n = corners.shape[0]
    
    P = chessboard_positions.copy();
    P[:,2] = 1
    
    Q = np.zeros((2*n, 9))
    Q[::2,0:3] = P
    Q[1::2,3:6] = P
    Q[::2,6:9] = -(P.T * corners[:,0]).T
    Q[1::2,6:9] = -(P.T * corners[:,1]).T
    
    U, D, Vh = np.linalg.svd(Q, full_matrices=True)
    # Row in vh that is associated with the smalles singular value gives solution
    H_tilde = Vh[-1,:]
    H = H_tilde.reshape(3,3)
    return H
    
def test_H_DLT():
    H_0 = calculate_H_DLT(image_corners[0], chessboard_positions)
    print(H_0)  
    
    # Check that perspective projection equation is approximately fulfilled
    P = chessboard_positions.copy();
    P[:,2] = 1
    
    pixels_h = H_0 @ P.T;
    pixels = pixels_h[0:2,:]/pixels_h[2,:]
    
    print("Average perspective projection equation error:", 
          np.average(np.linalg.norm(pixels.T - image_corners[0], axis=1)), 'px')

test_H_DLT()

[[7.11077081e-03 1.46102659e-04 2.71518127e-01]
 [3.97549542e-04 7.06366424e-03 9.62380051e-01]
 [3.58815984e-07 2.88318579e-07 1.38076544e-03]]
Average perspective projection equation error: 1.0310488434658875 px


We can now compute the homography $H_i$ for each image $i$. Note that the scale of $H$ can be arbitrary chosen beause the homogeneous equation $Q\cdot sH'=0$ is satisfied for any $s\in\mathbb{R}$. In the next step we need to extract the camera matrix from all matrices $H_i$. We will use the constraints imposed by the rotation matrix and rewrite the vectors $r_1$ and $r_2$ using $H$ and $K$ (and an unknown scale factor $s$). The vectors $g_j$ are the column vectors of $H$ and are not to be confused with the row vectors $h_j$ used in the derivation of $H$.
$$ H = \left[ \begin{array}{c c c} g_1 & g_2 & g_3 \end{array}\right] = 
sK\left[ \begin{array}{c c c} r_1 & r_2 & t \end{array}\right] \Rightarrow \left\{
\begin{array}{l l} r_1 = s^{-1} K^{-1}g_1 \\ r_2 = s^{-1} K^{-1}g_2 \end{array}\right.$$
The constraints on the rotation matrix impose that $r_1$ and $r_2$ are orthonormal which translates to:
$$ 
\begin{align}
    r_1 \cdot r_2 = 0 & \Rightarrow g_1^TK^{-T}K^{-1}g_2 = 0 \\
    r_1 \cdot r_1 = 1 = r_2 \cdot r_2 & \Rightarrow g_1^TK^{-T}K^{-1}g_1 = g_2^TK^{-T}K^{-1}g_2.
\end{align}
$$

We define the symmetric matrix $B$ which is defined by the vector $b'= \left( \begin{array}{llllll}
    b_{11} & b_{12} & b_{22} & b_{13}  & b_{23} & b_{33} \end{array}\right)^T$ as
$$ B = \left( \begin{array}{ccc}
    b_{11} & b_{12} & b_{13}\\ b_{12} & b_{22} & b_{23} \\ b_{13} & b_{23} & b_{33}
\end{array}\right)  = K^{-T}K^{-1}.$$
Using this noteation, the constraints can be rewritten as linear function of $b'$:
$$ g_i^T B g_j = v_{ij}^Tb' $$
with 
$$ v_{ij} = \left( \begin{array}{llllll} h_{1,i}h_{1,j} & h_{1,i}h_{2,j}+h_{2,i}h_{1,j} & h_{2,i}h_{2,j} & h_{3,i}h_{1,j}+h_{1,i}h_{3,j} & h_{3,i}h_{2,j}+h_{2,i}h_{3,j} & h_{3,i}h_{3,j}\end{array}\right)^T.$$
The corresponding two homogeneous equation in $b'$ are:
$$ \left( \begin{array}{c}
v_{12}^T \\ (v_{11}-v_{22})^T
\end{array}\right) b' = 0. $$
For each image $i$ we obtain two homogeneous equations that we stack to the matrix $V$. $b'$ is found again with the SVD.
$$ Vb'=0$$

In [5]:
def calculate_B_DLT(image_corners, chessboard_positions):
    def v(i, j, H):
        # Note the 0 indexing in numpy
        return np.array([H[0,i]*H[0,j], 
                         H[0,i]*H[1,j] + H[1,i]*H[0,j], 
                         H[1,i]*H[1,j], 
                         H[2,i]*H[0,j] + H[0,i]*H[2,j], 
                         H[2,i]*H[1,j] + H[1,i]*H[2,j],
                         H[2,i]*H[2,j]])
    
    n = image_corners.shape[0]
    V = np.zeros((2*n, 6))
    
    for i in range(n):
        H = calculate_H_DLT(image_corners[i], chessboard_positions)
        V[2*i,:] = v(0, 1, H)
        V[2*i+1,:] = v(0, 0, H) - v(1, 1, H)
        
    U, D, Vh = np.linalg.svd(V, full_matrices=True)
    b = Vh[-1,:]
    B = np.array([[b[0], b[1], b[3]],
                  [b[1], b[2], b[4]],
                  [b[3], b[4], b[5]]])
    return B
B = calculate_B_DLT(image_corners, chessboard_positions)
print(B)

[[ 2.58805673e-07 -9.94081941e-10 -2.60286939e-04]
 [-9.94081941e-10  2.51578496e-07 -1.86041522e-04]
 [-2.60286939e-04 -1.86041522e-04  9.99999949e-01]]


We can finally recover the calibration matrix $K$ from $B$ using the **Cholesky Factorization**. Keep in mind that we can only estimate $B$ up to a constant scale factor $\alpha$. Therefore $B=\alpha K^{-T}K^{-1}.$ We can safely ignore the scale factor at first and later rescale the obtained matrix $K$ so that $K_{33} =1$. Recall that the Cholesky Factorization factorizes a square matrix $A$ in the product of a lower-triangular matrix $L$ and a upper-triangular matrix $L^T$: $A=LL^T$. We apply this to $B$:
$$ B = LL^T = K'^{-T}K'^{-1} \Rightarrow K'=L^{-T} $$
The camera matrix $K$ can now be recovered as:
$$ K = \frac{1}{K'_{33}}K' $$

In [7]:
def B_to_K(B):
    L = np.linalg.cholesky(B)
    K_p = np.linalg.inv(L).T
    K = K_p/K_p[2,2]
    
    # Due to numeric imprecisions elements below the diaglonal are note exactely 0. We set then to 0 to be consistent.
    K[1,0] = K[2,0] = K[2,1] = 0
    return K

camera_mat = B_to_K(B)
with np.printoptions(suppress=True, precision=2):
    print("Our initial guess for the camera matrix")
    print(camera_mat)
    print("Camera matrix obtained by OpenCV")
    print(camera_mat_cv)

Our initial guess for the camera matrix
[[1521.55    5.93 1008.58]
 [   0.   1543.26  743.48]
 [   0.      0.      1.  ]]
Camera matrix obtained by OpenCV
[[1534.97    0.   1010.01]
 [   0.   1558.1   747.42]
 [   0.      0.      1.  ]]


As we can see we are already pretty close to the OpenCV solution which is a good sign! :) We now need to estimate the distortion coefficients and refine our estimate with non-linear optimization.
### Distortion Coefficients
One could try to estimate the distortion coefficients by using a least squares approach as described in Zhang's paper to obtain an initial guess. This guess can then be refined with non-linear optimization. In practice however one can often skip this step and initialize the refinement with all distortion coefficients set to 0.
#### Reprojection Error
The reprojection error measures the euclidian distance in pixels between the observed corner position and the position obtained by projecting the point using $K$ and the distortion coefficients.

In [8]:
def reprojection_error(corners, chessboard_positions):
    pass